# IBM Applied Data Science Capstone
## Segmenting and Clustering Neighborhoods in Toronto

### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import json

from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium # map rendering library
print("Libraries imported.")

Libraries imported.


## Scrapping data from Wikipedia Page into dataFrame

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [26]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
read_table = pd.read_html(url,header=[0])
toronto_df = read_table[0]
toronto_df.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

### Ignore cells with a borough that is Not assigned

In [30]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
toronto_df = toronto_df.replace('\n',' ', regex=True)
toronto_df = toronto_df[~toronto_df.Borough.str.contains('Not assigned')].reset_index(drop=True)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 

### Applying Groupby

In [31]:
toronto_df = toronto_df.groupby('Postal Code',as_index=False).agg(lambda x: ','.join(set(x.dropna())))
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [5]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Shape of dataframe 

In [33]:
toronto_df.shape

(103, 3)

### Sample dataframe

In [36]:
test_list = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L','M5V','M1B', 'M5A']
test_df = toronto_df[toronto_df['Postal Code'].isin(test_list)==True].reset_index(drop=True)
test_df

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1J,Scarborough,Scarborough Village
2,M1R,Scarborough,"Wexford, Maryvale"
3,M2H,North York,Hillcrest Village
4,M4B,East York,"Parkview Hill, Woodbine Gardens"
5,M4G,East York,Leaside
6,M4M,East Toronto,Studio District
7,M5A,Downtown Toronto,"Regent Park, Harbourfront"
8,M5G,Downtown Toronto,Central Bay Street
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


<hr>

## Downloading Geospatial_data of Toronto

In [38]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Combining Geospatial data  & Neighborhoods in  Toronto

In [41]:
combined = toronto_df.set_index('Postal Code').join(geo_df.set_index('Postal Code'))
combined.reset_index(inplace = True)
combined.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [25]:
test_list = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L','M5V','M1B', 'M5A']
test_df = combined[combined['Postal Code'].isin(test_list)==True].reset_index(drop=True)
test_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
5,M4G,East York,Leaside,43.709060,-79.363452
6,M4M,East Toronto,Studio District,43.659526,-79.340923
7,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
